# Relay Optimization Strategies

This notebook analyzes relay-controlled heating system optimization opportunities:
- Peak demand reduction through relay coordination
- Load shifting to match PV production
- Predictive control based on weather forecasts
- Integration with price-based optimization

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configure display options
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Import PEMS v2 modules
import sys
sys.path.append('../../..')  # Adjust path as needed

from pems_v2.analysis.data_extraction import DataExtractor
from pems_v2.analysis.data_preprocessing import RelayDataProcessor
from pems_v2.analysis.thermal_analysis import ThermalAnalyzer
from pems_v2.analysis.visualization import AnalysisVisualizer

## 1. Load Current Relay System Data

In [ ]:
# Initialize components
extractor = DataExtractor()
relay_processor = RelayDataProcessor()
thermal_analyzer = ThermalAnalyzer()
visualizer = AnalysisVisualizer()

# Extract recent data for analysis
end_date = datetime.now()
start_date = end_date - timedelta(days=90)  # 3 months for detailed analysis

print(f"Analysis period: {start_date.date()} to {end_date.date()}")

# Extract all data
all_data = extractor.extract_all_data(
    start_time=start_date.isoformat(),
    end_time=end_date.isoformat()
)

# Process relay data
room_data = all_data.get('rooms', {})
relay_analysis = relay_processor.process_relay_data(room_data)

# Display current system performance
if 'system_totals' in relay_analysis:
    system = relay_analysis['system_totals']
    print("\n=== Current System Performance ===")
    print(f"Peak demand: {system.get('peak_demand_kw', 0):.1f} kW")
    print(f"Average demand: {system.get('average_demand_kw', 0):.1f} kW")
    print(f"Total capacity: {system.get('total_installed_capacity_kw', 0):.1f} kW")
    print(f"Load factor: {system.get('load_factor', 0):.1%}")
    print(f"Diversity factor: {system.get('diversity_factor', 0):.2f}")
    print(f"Peak hour: {system.get('peak_hour', 'N/A')}:00")
    
    # Calculate potential savings
    current_peak = system.get('peak_demand_kw', 0)
    potential_peak = current_peak * 0.8  # 20% reduction target
    print(f"\nOptimization potential:")
    print(f"Target peak demand: {potential_peak:.1f} kW")
    print(f"Potential reduction: {current_peak - potential_peak:.1f} kW")

## 2. Peak Demand Analysis

In [ ]:
# Analyze peak demand patterns
total_power_data = []

# Reconstruct total power consumption timeline
for room_name, room_analysis in relay_analysis.items():
    if room_name != 'system_totals' and 'power_consumption' in room_analysis:
        power_series = room_analysis['power_consumption']
        if total_power_data is None:
            total_power_data = power_series.to_frame(name=room_name)
        else:
            total_power_data = total_power_data.join(power_series.to_frame(name=room_name), how='outer')

if len(total_power_data) > 0:
    # Calculate total system power
    total_power_data['total_power_kw'] = total_power_data.sum(axis=1) / 1000
    
    # Analyze peak events
    peak_threshold = total_power_data['total_power_kw'].quantile(0.95)  # Top 5% of demand
    peak_events = total_power_data[total_power_data['total_power_kw'] >= peak_threshold]
    
    print(f"\n=== Peak Demand Events ===")
    print(f"Peak threshold: {peak_threshold:.1f} kW")
    print(f"Number of peak events: {len(peak_events)}")
    
    # Identify rooms contributing to peaks
    room_contributions = {}
    for room in total_power_data.columns[:-1]:  # Exclude total_power_kw
        room_contributions[room] = (peak_events[room] > 0).sum() / len(peak_events) * 100
    
    # Sort by contribution
    sorted_contributions = sorted(room_contributions.items(), key=lambda x: x[1], reverse=True)
    
    print("\nRooms active during peak events:")
    for room, pct in sorted_contributions[:10]:  # Top 10 contributors
        print(f"  {room}: {pct:.1f}%")
    
    # Visualize peak patterns
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Daily peak pattern
    daily_peaks = total_power_data.resample('D')['total_power_kw'].max()
    axes[0, 0].plot(daily_peaks.index, daily_peaks.values)
    axes[0, 0].axhline(y=peak_threshold, color='r', linestyle='--', label='95th percentile')
    axes[0, 0].set_title('Daily Peak Demand')
    axes[0, 0].set_ylabel('Peak Power (kW)')
    axes[0, 0].legend()
    
    # Hourly pattern
    hourly_avg = total_power_data.groupby(total_power_data.index.hour)['total_power_kw'].mean()
    hourly_max = total_power_data.groupby(total_power_data.index.hour)['total_power_kw'].max()
    axes[0, 1].plot(hourly_avg.index, hourly_avg.values, label='Average', linewidth=2)
    axes[0, 1].plot(hourly_max.index, hourly_max.values, label='Maximum', linewidth=2)
    axes[0, 1].set_title('Hourly Demand Pattern')
    axes[0, 1].set_xlabel('Hour of Day')
    axes[0, 1].set_ylabel('Power (kW)')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Weekly pattern
    weekday_avg = total_power_data.groupby(total_power_data.index.weekday)['total_power_kw'].mean()
    axes[1, 0].bar(range(7), weekday_avg.values)
    axes[1, 0].set_title('Average Demand by Weekday')
    axes[1, 0].set_xlabel('Day of Week')
    axes[1, 0].set_ylabel('Average Power (kW)')
    axes[1, 0].set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
    
    # Room contribution to peaks
    top_contributors = dict(sorted_contributions[:8])
    axes[1, 1].bar(range(len(top_contributors)), list(top_contributors.values()))
    axes[1, 1].set_title('Room Contribution to Peak Events')
    axes[1, 1].set_xlabel('Room')
    axes[1, 1].set_ylabel('Active During Peaks (%)')
    axes[1, 1].set_xticklabels(list(top_contributors.keys()), rotation=45, ha='right')
    
    plt.tight_layout()
    plt.show()

## 3. Relay Coordination Opportunities

In [ ]:
# Analyze simultaneous relay operation
if len(total_power_data) > 0:
    # Count simultaneous relays
    relay_count = (total_power_data.iloc[:, :-1] > 0).sum(axis=1)
    
    # Analyze correlation between rooms
    room_correlations = total_power_data.iloc[:, :-1].corr()
    
    print("=== Relay Coordination Analysis ===")
    print(f"\nSimultaneous relay statistics:")
    print(f"Maximum simultaneous: {relay_count.max()} relays")
    print(f"Average simultaneous: {relay_count.mean():.1f} relays")
    print(f"95th percentile: {relay_count.quantile(0.95):.0f} relays")
    
    # Find highly correlated room pairs (candidates for staggering)
    high_correlations = []
    for i in range(len(room_correlations.columns)):
        for j in range(i+1, len(room_correlations.columns)):
            corr_value = room_correlations.iloc[i, j]
            if corr_value > 0.7:  # High correlation threshold
                high_correlations.append({
                    'Room 1': room_correlations.columns[i],
                    'Room 2': room_correlations.columns[j],
                    'Correlation': corr_value
                })
    
    if high_correlations:
        print("\nHighly correlated room pairs (coordination candidates):")
        corr_df = pd.DataFrame(high_correlations)
        corr_df = corr_df.sort_values('Correlation', ascending=False)
        print(corr_df.head(10).to_string(index=False))
    
    # Visualize correlation matrix
    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(room_correlations, dtype=bool))
    sns.heatmap(room_correlations, mask=mask, cmap='coolwarm', center=0,
                square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
    plt.title('Room Relay Operation Correlation Matrix')
    plt.tight_layout()
    plt.show()
    
    # Calculate potential peak reduction through coordination
    # Simulate staggered operation for top correlated pairs
    if high_correlations:
        # Simple simulation: prevent simultaneous operation of highly correlated rooms
        simulated_power = total_power_data.copy()
        
        for pair in high_correlations[:5]:  # Top 5 pairs
            room1, room2 = pair['Room 1'], pair['Room 2']
            # When both are on, delay one by 30 minutes
            both_on = (simulated_power[room1] > 0) & (simulated_power[room2] > 0)
            # Shift room2 operation when conflict exists
            simulated_power.loc[both_on, room2] = 0
        
        # Recalculate total with coordination
        simulated_power['coordinated_total_kw'] = simulated_power.iloc[:, :-2].sum(axis=1) / 1000
        
        original_peak = total_power_data['total_power_kw'].max()
        coordinated_peak = simulated_power['coordinated_total_kw'].max()
        
        print(f"\n=== Coordination Impact ===")
        print(f"Original peak: {original_peak:.1f} kW")
        print(f"Coordinated peak: {coordinated_peak:.1f} kW")
        print(f"Peak reduction: {original_peak - coordinated_peak:.1f} kW ({(original_peak - coordinated_peak)/original_peak*100:.1f}%)")

## 4. PV Integration Opportunities

In [ ]:
# Analyze potential for shifting load to PV production hours
pv_data = all_data.get('pv', pd.DataFrame())
weather_data = all_data.get('weather', pd.DataFrame())

if not pv_data.empty and len(total_power_data) > 0:
    print("=== PV Integration Analysis ===")
    
    # Align PV and relay data
    analysis_df = pd.DataFrame({
        'relay_demand_kw': total_power_data['total_power_kw'],
        'pv_production_kw': pv_data.get('InputPower', 0) / 1000
    })
    
    # Calculate hourly patterns
    hourly_patterns = analysis_df.groupby(analysis_df.index.hour).mean()
    
    # Find mismatch between production and consumption
    hourly_patterns['mismatch_kw'] = hourly_patterns['relay_demand_kw'] - hourly_patterns['pv_production_kw']
    
    # Identify load shifting opportunities
    pv_hours = hourly_patterns[hourly_patterns['pv_production_kw'] > 1.0].index  # Significant PV production
    non_pv_hours = hourly_patterns[hourly_patterns['pv_production_kw'] <= 1.0].index
    
    current_pv_hour_load = hourly_patterns.loc[pv_hours, 'relay_demand_kw'].sum()
    current_non_pv_hour_load = hourly_patterns.loc[non_pv_hours, 'relay_demand_kw'].sum()
    
    print(f"\nCurrent load distribution:")
    print(f"During PV hours (typically {pv_hours[0]}-{pv_hours[-1]}h): {current_pv_hour_load:.1f} kW")
    print(f"During non-PV hours: {current_non_pv_hour_load:.1f} kW")
    print(f"PV hour utilization: {current_pv_hour_load/(current_pv_hour_load+current_non_pv_hour_load)*100:.1f}%")
    
    # Visualize load shifting potential
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
    
    # Current pattern
    ax1.bar(hourly_patterns.index, hourly_patterns['relay_demand_kw'], 
            alpha=0.7, label='Relay Demand', color='red')
    ax1.plot(hourly_patterns.index, hourly_patterns['pv_production_kw'], 
             'g-', linewidth=3, label='PV Production')
    ax1.set_title('Current Hourly Energy Pattern')
    ax1.set_ylabel('Power (kW)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Optimized pattern (simulation)
    optimized_demand = hourly_patterns['relay_demand_kw'].copy()
    
    # Simple optimization: shift 30% of non-PV hour load to PV hours
    shift_amount = hourly_patterns.loc[non_pv_hours, 'relay_demand_kw'].mean() * 0.3
    optimized_demand[non_pv_hours] -= shift_amount
    optimized_demand[pv_hours] += shift_amount * len(non_pv_hours) / len(pv_hours)
    
    ax2.bar(hourly_patterns.index, optimized_demand, 
            alpha=0.7, label='Optimized Demand', color='blue')
    ax2.plot(hourly_patterns.index, hourly_patterns['pv_production_kw'], 
             'g-', linewidth=3, label='PV Production')
    ax2.set_title('Optimized Hourly Energy Pattern (30% Load Shift)')
    ax2.set_xlabel('Hour of Day')
    ax2.set_ylabel('Power (kW)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate self-consumption improvement
    current_self_consumption = np.minimum(hourly_patterns['relay_demand_kw'], 
                                         hourly_patterns['pv_production_kw']).sum()
    optimized_self_consumption = np.minimum(optimized_demand, 
                                           hourly_patterns['pv_production_kw']).sum()
    
    print(f"\n=== Self-Consumption Improvement ===")
    print(f"Current self-consumption: {current_self_consumption:.1f} kWh/day")
    print(f"Optimized self-consumption: {optimized_self_consumption:.1f} kWh/day")
    print(f"Improvement: {optimized_self_consumption - current_self_consumption:.1f} kWh/day ")
    print(f"            ({(optimized_self_consumption/current_self_consumption - 1)*100:.1f}% increase)")

## 5. Thermal Analysis for Predictive Control

In [ ]:
# Analyze thermal characteristics for predictive control
if room_data and weather_data is not None and not weather_data.empty:
    print("=== Thermal Analysis for Predictive Control ===")
    
    # Run thermal analysis
    thermal_results = thermal_analyzer.analyze_room_dynamics(room_data, weather_data)
    
    # Extract RC parameters for key rooms
    rc_summary = []
    
    for room_name, room_results in thermal_results.items():
        if isinstance(room_results, dict) and 'rc_model' in room_results:
            rc_model = room_results['rc_model']
            if 'thermal_resistance' in rc_model:
                rc_summary.append({
                    'Room': room_name,
                    'R (°C/W)': rc_model.get('thermal_resistance', 0),
                    'C (Wh/°C)': rc_model.get('thermal_capacity', 0),
                    'Time Constant (h)': rc_model.get('time_constant', 0),
                    'Model R²': rc_model.get('r_squared', 0)
                })
    
    if rc_summary:
        rc_df = pd.DataFrame(rc_summary)
        rc_df = rc_df.sort_values('Time Constant (h)', ascending=False)
        print("\nThermal characteristics by room:")
        print(rc_df.head(10).to_string(index=False))
        
        # Identify rooms suitable for pre-heating
        slow_rooms = rc_df[rc_df['Time Constant (h)'] > 2.0]  # Slow thermal response
        fast_rooms = rc_df[rc_df['Time Constant (h)'] <= 1.0]  # Fast thermal response
        
        print(f"\n=== Predictive Control Recommendations ===")
        print(f"Rooms suitable for pre-heating (slow response): {len(slow_rooms)}")
        if not slow_rooms.empty:
            print("Top candidates:")
            for _, room in slow_rooms.head(5).iterrows():
                print(f"  - {room['Room']}: τ = {room['Time Constant (h)']:.1f}h")
        
        print(f"\nRooms requiring just-in-time heating (fast response): {len(fast_rooms)}")
        if not fast_rooms.empty:
            print("Top candidates:")
            for _, room in fast_rooms.head(5).iterrows():
                print(f"  - {room['Room']}: τ = {room['Time Constant (h)']:.1f}h")
        
        # Visualize thermal parameters
        if len(rc_df) > 5:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
            
            # Time constants
            top_rooms = rc_df.head(10)
            ax1.barh(range(len(top_rooms)), top_rooms['Time Constant (h)'])
            ax1.set_yticks(range(len(top_rooms)))
            ax1.set_yticklabels(top_rooms['Room'])
            ax1.set_xlabel('Time Constant (hours)')
            ax1.set_title('Thermal Time Constants by Room')
            ax1.axvline(x=2.0, color='r', linestyle='--', alpha=0.5, label='Pre-heat threshold')
            ax1.legend()
            
            # Model quality
            ax2.scatter(rc_df['Time Constant (h)'], rc_df['Model R²'])
            ax2.set_xlabel('Time Constant (hours)')
            ax2.set_ylabel('Model R²')
            ax2.set_title('Model Quality vs Time Constant')
            ax2.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()

## 6. Optimization Strategy Summary

In [ ]:
# Compile optimization recommendations
print("=== RELAY OPTIMIZATION STRATEGY SUMMARY ===")

strategies = []

# Peak reduction strategy
if 'system_totals' in relay_analysis:
    current_peak = relay_analysis['system_totals'].get('peak_demand_kw', 0)
    if current_peak > 15:  # Significant peak
        strategies.append({
            'Strategy': 'Peak Demand Reduction',
            'Method': 'Relay coordination and staggering',
            'Potential': f'{current_peak * 0.2:.1f} kW reduction',
            'Implementation': 'Prevent simultaneous operation of correlated rooms'
        })

# Load shifting strategy
if not pv_data.empty:
    strategies.append({
        'Strategy': 'PV Self-Consumption',
        'Method': 'Shift heating loads to PV production hours',
        'Potential': '30% increase in self-consumption',
        'Implementation': 'Pre-heat during midday, reduce evening heating'
    })

# Predictive control strategy
if rc_summary:
    slow_room_count = len(rc_df[rc_df['Time Constant (h)'] > 2.0])
    if slow_room_count > 0:
        strategies.append({
            'Strategy': 'Predictive Pre-heating',
            'Method': 'Weather-based anticipatory control',
            'Potential': f'{slow_room_count} rooms with slow response',
            'Implementation': 'Start heating 2-3 hours before occupancy'
        })

# Price-based optimization
if 'prices' in all_data:
    strategies.append({
        'Strategy': 'Price-Based Optimization',
        'Method': 'Shift loads to low-price periods',
        'Potential': '15-20% cost reduction',
        'Implementation': 'Integrate with electricity price forecasts'
    })

# Display strategies
if strategies:
    strategy_df = pd.DataFrame(strategies)
    print(strategy_df.to_string(index=False))
    
    # Calculate combined impact
    print("\n=== Combined Optimization Impact ===")
    print("Estimated benefits from full implementation:")
    print(f"- Peak demand reduction: 20-25%")
    print(f"- Energy cost savings: 15-25%")
    print(f"- PV self-consumption increase: 30-40%")
    print(f"- Comfort improvement through predictive control")
    print(f"- Extended relay lifetime through reduced switching")

# Implementation roadmap
print("\n=== Implementation Roadmap ===")
print("Phase 1 (Immediate):")
print("- Implement basic relay coordination rules")
print("- Set maximum simultaneous relay limit")
print("- Adjust heating schedules to PV production hours")

print("\nPhase 2 (1-3 months):")
print("- Deploy predictive control for slow-response rooms")
print("- Integrate weather forecast data")
print("- Implement price-based optimization")

print("\nPhase 3 (3-6 months):")
print("- Machine learning for occupancy prediction")
print("- Advanced optimization with battery storage")
print("- Full system integration and automation")